In [132]:
import spglib
import numpy as np
from pyxtal.lattice import para2matrix

def generate_lattice_vectors(
    lattice_type, a, b=None, c=None, alpha=90, beta=90, gamma=90
):
    """
    Generates lattice vectors for the given lattice type and parameters.

    Parameters:
    lattice_type (str): Type of lattice ("P", "F", "B", "CXY", "CYZ", "CXZ", "R", "H")
    a, b, c (float): Lattice parameters
    alpha, beta, gamma (float): Angles in degrees

    Returns:
    numpy.ndarray: Lattice vectors
    """
    # Convert angles from degrees to radians
    alpha = np.radians(alpha)
    beta = np.radians(beta)
    gamma = np.radians(gamma)

    sin = np.sin
    cos = np.cos
    arccos = np.arccos

    gamma_dash = arccos(
        (cos(gamma) - cos(beta) * cos(alpha)) / (sin(alpha) * sin(beta))
    )

    # Default b and c to a if they are not provided
    b = b if b is not None else a
    c = c if c is not None else a

    if lattice_type == "P":
        # Primitive lattice
        lattice = np.array(
            [
                a
                * [
                    sin(gamma_dash) * sin(beta),
                    cos(gamma_dash) * cos(beta),
                    cos(beta),
                ],
                b * [0, sin(alpha), cos(alpha)],
                [0, 0, c],
            ]
        )

    elif lattice_type == "F":
        # Face-centered lattice
        lattice = np.array([[0, b / 2, c / 2], [a / 2, 0, c / 2], [a / 2, b / 2, 0]])

    elif lattice_type == "B":
        # Body-centered lattice
        lattice = np.array(
            [
                [-a / 2, b / 2, c / 2],
                [a / 2, -b / 2, c / 2],
                [a / 2, b / 2, -c / 2],
            ]
        )

    elif lattice_type == "CXY":
        # C-base-centered orthorhombic lattice
        lattice = np.array([[a / 2, -b / 2, 0], [a / 2, b / 2, 0], [0, 0, c]])

    elif lattice_type == "CYZ":
        # A-base-centered orthorhombic lattice
        lattice = np.array([[a, 0, 0], [0, -b / 2, c / 2], [0, b / 2, c / 2]])

    elif lattice_type == "CXZ":
        # B-base-centered orthorhombic or monoclinic lattice
        lattice = np.array(
            [
                [a * sin(gamma) / 2, a * cos(gamma) / 2, -c / 2],
                [0, b, 0],
                [a * sin(gamma) / 2, a * cos(gamma) / 2, c / 2],
            ]
        )

    elif lattice_type == "R":
        # Rhombohedral lattice
        lattice = np.array(
            [
                [a/np.sqrt(3)/2, -a/2, c/3],
                [a/np.sqrt(3)/2, a/2, c/3],
                [-a/np.sqrt(3), 0, c/3]
            ]
        )

    elif lattice_type == "H":
        # Hexagonal lattice
        lattice = np.array([
            [np.sqrt(3)*a/2, -a/2, 0],
            [0, a, 0],
            [0, 0, c]
        ])

    else:
        raise ValueError("Unknown lattice type: {}".format(lattice_type))

    return lattice


def get_symm_ops(
    a, b, c, alpha, beta, gamma, atomic_positions, atomic_numbers
):
    """
    Determine the space group of a system given its cell dimensions, cell angles, and atomic positions.

    Parameters:
    a, b, c (float): Cell dimensions.
    alpha, beta, gamma (float): Cell angles in degrees.
    atomic_positions (list of list of float): Atomic positions in fractional coordinates.
    atomic_numbers (list of int): Atomic numbers corresponding to the atoms in the system.

    Returns:
    str: Space group symbol.
    int: Space group number.
    """
    # Convert angles from degrees to radians
    alpha = np.radians(alpha)
    beta = np.radians(beta)
    gamma = np.radians(gamma)

    lattice = para2matrix([a, b, c, alpha, beta, gamma])
    print(lattice)
    
    # Create the cell
    cell = (lattice, atomic_positions, atomic_numbers)

    # Use spglib to determine the space group
    # cell = spglib.find_primitive(cell)
    symmetry_ops = spglib.get_symmetry(cell, symprec=1e-5)

    space_group_info = spglib.get_spacegroup(cell, symprec=1e-5)
    space_group_symbol, space_group_number = space_group_info.split("(")[0], int(
        space_group_info.split(")")[0].split("(")[1]
    )
    print(space_group_symbol, space_group_number)

    return symmetry_ops

In [133]:
# Example usage
a, b, c = 8.037761, 8.037761, 27.667480
alpha, beta, gamma = 90.000000, 90.000000, 90.000000
atomic_positions = [
    [0.00000000, 0.00000000, 0.58290000],
    [0.00000000, 0.50000000, 0.83290000],
    [0.00000000, 0.00000000, 0.16860000],
    [0.00000000, 0.50000000, 0.41860000],
    [0.00000000, 0.00000000, 0.00000000],
    [0.00000000, 0.50000000, 0.25000000]
]
atomic_numbers = [59, 59, 13, 13, 32, 32]
# atomic_numbers = [1, 2, 3]

o = get_symm_ops(a, b, c, alpha, beta, gamma, atomic_positions, atomic_numbers)
print(o["rotations"])
print(o["translations"])
print(o["equivalent_atoms"])

[[8.03776100e+00 4.92170914e-16 4.92170914e-16]
 [0.00000000e+00 8.03776100e+00 4.92170914e-16]
 [0.00000000e+00 0.00000000e+00 2.76674800e+01]]
Pmm2  25
[[[ 1  0  0]
  [ 0  1  0]
  [ 0  0  1]]

 [[-1  0  0]
  [ 0 -1  0]
  [ 0  0  1]]

 [[-1  0  0]
  [ 0  1  0]
  [ 0  0  1]]

 [[ 1  0  0]
  [ 0 -1  0]
  [ 0  0  1]]]
[[0. 0. 0.]
 [0. 0. 0.]
 [0. 0. 0.]
 [0. 0. 0.]]
[0 1 2 3 4 5]


In [134]:
positions = [
    [0,	0,	0.181698],
	[1/3,	2/3,	0.18035],
	[0.069572,	0.534786,	3/4],
	[0.202348,	0.404696,	3/4],
	[0.084276,	0.542138,	0.135217],
	[0.12355,	0.247099,	0.638691],
	[0.04994,	0.719356,	0.435477]
] # Si
numbers = [1] * 7

symm_ops = get_symm_ops("H", 10.37, 10.37, 16.97, 90, 90, 120, positions, numbers)
print(symm_ops["rotations"].__len__())

TypeError: get_symm_ops() takes 8 positional arguments but 9 were given